## Imports
all the requirements for the project. This includes own code. 

In [1]:
import dates_generation as dg
import os.path
import pandas as pd
import xlsxwriter
import dataframe_class_base as dcb
import sqlite_db_generation as sqlgen
import sqlite3
import math
import random
import main_process_functions as mpf

## Dates generation

Generating dictionary with business days where the counting will take place and taking into account the holidays of the month


In [2]:
holidays = ["05-05-23", "04-05-23"]

In [3]:
dates_dictionary = dg.date_generation_process(5, 2023, holidays)

## Dataframe base ABC clasification

Loading ABC base file and generating dataframe with abc clasificacion and the quantity that we have to count for each product

In [4]:
# read ABC base file whith products and its clasification
fullpath = os.path.abspath(".//abc_mayo_23.xlsx")

df_clasificacion = pd.read_excel(fullpath, sheet_name="Hoja1")

In [5]:
df_abc = dcb.cantidad_segun_abc(df_clasificacion)

## DB Operations

Database creation. Testing of database related functions such as creating database and inserting dataframe rows to that database. Removing quantity from article, selecting random article  and checking if article has any quantity available for counting yet.


In [6]:
sqlgen.create_db("abc_quantity.db", "abc")
sqlgen.insert_article_df_into_db(df_abc, "abc_quantity.db", "abc")



In [7]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [8]:
sqlgen.select_an_article("abc_quantity.db", "abc", 25668)

(25668, 'PEPSI NF PET X6 500CC', 'A', 3)


In [9]:
random_art = sqlgen.select_random_article("abc_quantity.db", "abc", "A")

In [10]:
sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", random_art[0])

True

In [11]:
a_articles_per_day = sqlgen.get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"A")
b_articles_per_day = sqlgen.get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"B")
c_articles_per_day = sqlgen.get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"C")


## Testing full cycle
To test full cycle we will restart the database, make the random selection of items for each day and assign to dataframe. Then there is a for-loop to check the amount of product each day to be able to verify the balance and a last check to assure the state of quantity in the database.

If the program works as expected the list should be quite balanced regarding amount of items to count each day and the last check should give *FALSE* status because there are no items left to be counted.

In [12]:
dates_dictionary = dg.date_generation_process(5, 2023, holidays)
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")


In [13]:
dictionary_output = mpf.Assignation_process(dates_dictionary, a_articles_per_day,b_articles_per_day,c_articles_per_day)

In [14]:
for i in dictionary_output:
    print(i, len(dictionary_output[i]))

01-05-2023 14
02-05-2023 14
03-05-2023 14
06-05-2023 14
08-05-2023 14
09-05-2023 14
10-05-2023 14
11-05-2023 12
12-05-2023 14
13-05-2023 14
15-05-2023 14
16-05-2023 14
17-05-2023 14
18-05-2023 14
19-05-2023 14
20-05-2023 14
22-05-2023 5
23-05-2023 11
24-05-2023 14
25-05-2023 14
26-05-2023 14
27-05-2023 14
29-05-2023 10
30-05-2023 14
31-05-2023 11


In [15]:
sqlgen.check_if_item_without_count("abc_quantity.db","abc")

False

## File saving
 Save dataframes to single excel file

In [17]:
file_name = "planilla_procesada"
writer = pd.ExcelWriter(file_name + ".xlsx", engine="xlsxwriter")


for i in dates_dictionary:
    dates_dictionary[i].to_excel(writer, sheet_name=i)
    workbook = writer.book
    worksheet = writer.sheets[i]
    cell_format = workbook.add_format({"border": 1})

    for j in range(140):
        worksheet.set_row(j, 25, cell_format)
        worksheet.set_column(j, width=10, last_col=12)
        if j == 1:
            worksheet.set_column(j, width=42, last_col=12)
writer.close()